In [31]:
from torch.utils.data import Dataset, DataLoader
import csv
import json
import re
import os

class Mydataset(Dataset):
    def __init__(self, train=True):
        if train:
            print('loading trainig dataset')
            file = 'train_poetry.csv'
        else:
            print('loading testing dataset')
            file = 'test_poetry.csv'
        with open(file, 'r', newline='', encoding='utf-8') as myFile:  
            rdr = csv.reader(myFile)
            temp = list(rdr)
            self.x = [a[0][:200] for a in temp]
            self.y = [a[1] for a in temp]
        self.len = len(self.x)
        self.labels = list(sorted(set(self.y)))
        self.alphabet = list(sorted(set(['s','w','\t'])))
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.len
        
    def get_labels(self):
        return self.labels
    
    def get_alphabet(self):
        return self.alphabet
    
    def get_alphabet_id(self, c):
        return self.alphabet.index(c)
    
    def get_label(self, id):
        return self.labels[id]
    
    def get_label_id(self, label):
        return self.labels.index(label)

In [32]:
train_dataset = Mydataset()
test_dataset = Mydataset(train=False)

loading trainig dataset
loading testing dataset


In [33]:
BATCH_SIZE = 128

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=False)

N_LABELS = len(train_dataset.get_labels())
N_ALPHABET = len(train_dataset.get_alphabet())

In [34]:
class args:
    pass

args = args()

args.class_num = N_LABELS
args.kernel_num = 100
args.kernel_sizes = [2,3,4,5]
args.dropout = 0.5
args.static = True
args.lr = 0.001
args.epochs = 256
args.embeding_num = N_ALPHABET
args.embeding_dim = N_ALPHABET

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class CNN_Text(nn.Module):
    
    def __init__(self, args):
        super(CNN_Text, self).__init__()
        self.args = args
        
        C = args.class_num
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes
        V = args.embeding_num
        D = args.embeding_dim
        
        self.embed = nn.Embedding(V, D)

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])

        self.dropout = nn.Dropout(args.dropout)

        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def forward(self, x):

        if self.args.static:
            x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)

        logit = self.fc1(x)  # (N, C)
        return logit
model = CNN_Text(args)
model

CNN_Text(
  (embed): Embedding(3, 3)
  (convs1): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 3), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 3), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 3), stride=(1, 1))
    (3): Conv2d(1, 100, kernel_size=(5, 3), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc1): Linear(in_features=400, out_features=2, bias=True)
)

In [39]:
args.log_interval = 10
args.test_interval = 50

def train(model, train_loader, test_loader, args):
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    model.train()
    
    steps = 0
    
    for epoch in range(1, args.epochs+1):
        batch = 0
        for x, y in train_loader:
            x_t, y_t = build_tensor(x, y)
            optimizer.zero_grad()
            logit = model(x_t)
            loss = F.cross_entropy(logit, y_t)
            loss.backward()
            optimizer.step()
            steps += 1
            if steps % args.log_interval == 0:
                corrects = (torch.max(logit, 1)[1].view(y_t.size()).data == y_t.data).sum()
                accuracy = 100.0 * corrects/y_t.shape[0]
                print(
                    '\rBatch[{}] - loss: {:.6f}  acc: {:.4f}%({}/{})'.format(steps,
                                                                             loss.data[0], 
                                                                             accuracy,
                                                                             corrects,
                                                                             y_t.shape[0]))
            if steps % args.test_interval == 0:
                acc = eval(test_loader, model, args)
                
def eval(test_loader, model, args):
    model.eval()
    corrects, avg_loss = 0, 0
    for x, y in test_loader:
        x_t, y_t = build_tensor(x, y)
        logit = model(x_t)
        loss = F.cross_entropy(logit, y_t, size_average=False)
        avg_loss += loss.data[0]
        corrects += (torch.max(logit, 1)[1].view(y_t.size()).data == y_t.data).sum()
    size = len(test_loader.dataset)
    avg_loss /= size
    accuracy = 100.0 * corrects/size
    print('\nEvaluation - loss: {:.6f}  acc: {:.4f}%({}/{}) \n'.format(avg_loss, 
                                                                       accuracy, 
                                                                       corrects, 
                                                                       size))
    return accuracy

In [40]:
def build_tensor(x, y):
    label_t = torch.tensor([train_dataset.get_label_id(l) for l in y], dtype=torch.long)
    
    x_length = max([len(a) for a in x])
    x_temp = []
    for poem in x:
        t = torch.zeros([x_length, N_ALPHABET])
        for i,a in enumerate(poem):
            t[i][train_dataset.get_alphabet_id(a)] = 1
        x_temp.append(t)
    x_temp = [a.view(1, x_length, N_ALPHABET) for a in x_temp]
    return torch.cat(x_temp, 0), label_t 

try:
    train(model, train_loader, test_loader, args)
except KeyboardInterrupt:
    print('\n' + '-' * 89)
    print('Exiting from training early')
    eval(test_loader, model, args)

F:\settings\anaconda\lib\site-packages\ipykernel\__main__.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Batch[10] - loss: 0.703199  acc: 56.0000%(72/128)
Batch[20] - loss: 0.697434  acc: 53.0000%(68/128)
Batch[30] - loss: 0.695099  acc: 53.0000%(43/80)
Batch[40] - loss: 0.690557  acc: 56.0000%(72/128)
Batch[50] - loss: 0.659776  acc: 59.0000%(76/128)


F:\settings\anaconda\lib\site-packages\ipykernel\__main__.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Evaluation - loss: 0.655568  acc: 68.0000%(143/208) 

Batch[60] - loss: 0.636766  acc: 73.0000%(59/80)
Batch[70] - loss: 0.656228  acc: 60.0000%(77/128)
Batch[80] - loss: 0.613737  acc: 78.0000%(100/128)
Batch[90] - loss: 0.626658  acc: 73.0000%(59/80)
Batch[100] - loss: 0.624308  acc: 67.0000%(86/128)

Evaluation - loss: 0.608310  acc: 74.0000%(154/208) 

Batch[110] - loss: 0.641470  acc: 60.0000%(78/128)
Batch[120] - loss: 0.609769  acc: 72.0000%(58/80)
Batch[130] - loss: 0.573395  acc: 78.0000%(100/128)
Batch[140] - loss: 0.570076  acc: 76.0000%(98/128)
Batch[150] - loss: 0.591563  acc: 73.0000%(59/80)

Evaluation - loss: 0.567704  acc: 73.0000%(153/208) 

Batch[160] - loss: 0.544241  acc: 76.0000%(98/128)
Batch[170] - loss: 0.601905  acc: 65.0000%(84/128)
Batch[180] - loss: 0.609185  acc: 68.0000%(55/80)
Batch[190] - loss: 0.584258  acc: 66.0000%(85/128)
Batch[200] - loss: 0.532224  acc: 78.0000%(101/128)

Evaluation - loss: 0.535509  acc: 73.0000%(152/208) 

Batch[210] - loss: 0.

Batch[1380] - loss: 0.465286  acc: 76.0000%(61/80)
Batch[1390] - loss: 0.414584  acc: 80.0000%(103/128)
Batch[1400] - loss: 0.450715  acc: 80.0000%(103/128)

Evaluation - loss: 0.469546  acc: 75.0000%(156/208) 

Batch[1410] - loss: 0.421013  acc: 80.0000%(64/80)
Batch[1420] - loss: 0.424568  acc: 78.0000%(101/128)
Batch[1430] - loss: 0.444514  acc: 78.0000%(101/128)
Batch[1440] - loss: 0.473696  acc: 76.0000%(61/80)
Batch[1450] - loss: 0.412411  acc: 82.0000%(106/128)

Evaluation - loss: 0.465991  acc: 75.0000%(157/208) 

Batch[1460] - loss: 0.381679  acc: 85.0000%(109/128)
Batch[1470] - loss: 0.442635  acc: 81.0000%(65/80)
Batch[1480] - loss: 0.491628  acc: 81.0000%(104/128)
Batch[1490] - loss: 0.431640  acc: 78.0000%(101/128)
Batch[1500] - loss: 0.364067  acc: 85.0000%(68/80)

Evaluation - loss: 0.466708  acc: 75.0000%(157/208) 

Batch[1510] - loss: 0.426157  acc: 80.0000%(103/128)
Batch[1520] - loss: 0.465911  acc: 79.0000%(102/128)
Batch[1530] - loss: 0.410881  acc: 83.0000%(67/80)